In [2]:
tcf(filename::String) = (@__DIR__) * "\\" * filename;

using Gridap
using GridapGmsh

# Исследование напряженного состояния кронштейна

В рамках данного урока м рассмотрим трехмерную задачу о нагружении кронштейна.
В прошлых уроках мы строили геометрию вручную при помощи инструментов gmsh, однако для более сложной геометрии такой подход не слишком удобен, и гораздо проще воспользоваться какой-либо CAD системой, позволяющей строить 3D геометрию.
Расчетная схема представлена на рисунке 1.

В данном случае, геометрия была построена в [бесплатном пакете FreeCAD]/[отечественном пакете Компас3D] и экспортирована в формат `*.stp`. Полученая геометрия показана на рисунке ниже.

Для приложения нагрузки цилиндрическая поверхность была разделена на две половины и нижня окрашена в цвет (0, 100, 0). В дальнейшем мы буем использовать цвет поверхности для быстрого поиска данной поверхности при загрузке ее gmsh. Аналогичным образом поверхность, которая фиксируется, помечена цветом (100, 0, 0).

## Импорт геометрии подготовка КЭ сетки

Импорт готовой геометрии в gmsh осуществляется при помощи функции `gmsh.model.occ.importShapes(file_name)`
Далее необходимо выявить индексы поверхностей, которые были помечены при моделировании. Для этого можно поместить их в словарь, ключем которого будет являтся цвет, а значением - индекс поверхности. После этого данным поверхностям следует задать метки, для чего необходимо поместить их в физический группы при помощи команды `gmsh.model.addPhysicalGroup(entity_dimension, vector_of_entities_tags, ph_group_id, "group_label")`. Кроме того, необходимо создать физическую группу для основного тела, для того чтобы в дальнейшем gmsh включил его в задание на построение сетки. 


In [1]:
using Gmsh

gmsh.initialize()

gmsh.model.add("model")

path = "lug.stp" |> tcf
v = gmsh.model.occ.importShapes(path)

gmsh.model.occ.synchronize()


faces = Dict()
for plane in gmsh.model.getEntities(2)
    faces[gmsh.model.getColor(plane...)] = plane[2];
    # gmsh.model.addPhysicalGroup(2, [plane[2]], -1, "$(plane[2])");
end



UndefVarError: UndefVarError: tcf not defined

Далее необходимо задать параметры для построения сетки. В Gmsh предусмотрено несколько вариантов для задания параметров сетки. Один из них - при мпомощи команды gmsh.options.setNumber("Parameter")В качестве параметров можно задать:
 - максимальный размер ст